In [1]:
import pandas as pd
from io import StringIO
import os

## Read trace

In [2]:
def read_trace(trace_dir, trace_file, worker_id=None):
    trace_filepath = os.path.join(trace_dir, trace_file)
    
    with open(trace_filepath,"r") as rf:
        new_trace = StringIO()
        line = rf.readline()
        while line:
            line_list = line.split()
            if line_list[0] == "[trace]":
                new_trace.write(line)
            line = rf.readline()
       
    new_trace.seek(0)
    df = pd.read_csv(new_trace, sep=" ", names=["InfoType", "EventType", "EventState", "ID", "Time"])
    del df["InfoType"]
    
    if worker_id is not None:
        df.insert(0, "WorkerID", worker_id)
        
    new_trace.close()
    
    return df

### COORD

In [3]:
trace_dir = "/home/posheng/billy_ws/fl_025/paslab/trace/async2_fl_sagg_3rm_mask_nossl/0901"
coord_trace_file = "coord_trace.txt"
hpc0_trace_file = "hpc0_trace.txt"
hpc1_trace_file = "hpc1_trace.txt"
hpc2_trace_file = "hpc2_trace.txt"
# hpc3_trace_file = "hpc3_trace.txt"

In [4]:
# coord_df = read_trace(trace_dir, coord_trace_file)
# hpc1_df = read_trace(trace_dir, hpc1_trace_file, "alice")
# hpc2_df = read_trace(trace_dir, hpc2_trace_file, "bob")
# hpc3_df = read_trace(trace_dir, hpc3_trace_file, "charlie")

coord_df = read_trace(trace_dir, coord_trace_file)
w0_df = read_trace(trace_dir, hpc0_trace_file, "alice")
w1_df = read_trace(trace_dir, hpc1_trace_file, "bob")
w2_df = read_trace(trace_dir, hpc2_trace_file, "charlie")

In [5]:
hpc1_df = w0_df
hpc2_df = w1_df
hpc3_df = w2_df

In [6]:
worker_dfs = {
    "alice": hpc1_df,
    "bob": hpc2_df,
    "charlie": hpc3_df
}

## Duration

In [7]:
def get_worker_duration(worker_id, worker_dfs, event_type):
    worker_df = worker_dfs[worker_id]
    worker_event_df = worker_df[worker_df["EventType"].str.contains(event_type, regex=False)]
    worker_event_df = worker_event_df.reset_index()
    
    del worker_event_df['index']
    del worker_event_df['ID']
    
    return worker_event_df

In [8]:
def get_coord_duration(coord_df, event_type):
    coord_event_df = coord_df[coord_df["EventType"].str.contains(event_type, regex=False)]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    del coord_event_df['ID']
    
    return coord_event_df

In [9]:
def calculate_duration(event_type, df):
    event_df = df[df["EventType"] == event_type][["EventType", "EventState", "ID", "Time"]]
    event_start_df = event_df[event_df["EventState"] == "start"][["ID", "Time"]]
    event_start_df = event_start_df.reset_index()

    event_end_df = event_df[event_df["EventState"] == "end"][["Time"]]
    event_end_df = event_end_df.reset_index()
    
    event_df_cat = pd.concat([event_start_df, event_end_df], axis=1)
    del event_df_cat['index']
    event_df_cat.columns = ["ID", "StartTime", "EndTime"]
    
    event_df_cat["Duration"] = event_df_cat.apply(lambda x: x["EndTime"] - x["StartTime"], axis=1)
    event_df_cat.insert(0, "EventType", event_type)
    return event_df_cat


In [10]:
GlobalModelSendDuration = calculate_duration("GlobalModelSend", coord_df)

In [11]:
LossFuncSendDuration = calculate_duration("LossFuncSend", coord_df)

In [12]:
ModelConfigSendDuration = calculate_duration("ModelConfigSend", coord_df)

### GlobalModelSendDuration

In [13]:
GlobalModelSendDuration[GlobalModelSendDuration["ID"] == "alice"]['Duration'].mean()

0.3205100893974304

In [14]:
GlobalModelSendDuration[GlobalModelSendDuration["ID"] == "bob"]['Duration'].mean()

0.3117477774620056

In [15]:
GlobalModelSendDuration[GlobalModelSendDuration["ID"] == "charlie"]['Duration'].mean()

0.31217079162597655

### LossFuncSendDuration

In [16]:
LossFuncSendDuration[LossFuncSendDuration["ID"] == "alice"]['Duration'].mean()

0.0012658357620239258

In [17]:
LossFuncSendDuration[LossFuncSendDuration["ID"] == "bob"]['Duration'].mean()

0.0012076854705810546

In [18]:
LossFuncSendDuration[LossFuncSendDuration["ID"] == "charlie"]['Duration'].mean()

0.0012090802192687988

### ModelConfigSendDuration

In [19]:
ModelConfigSendDuration[ModelConfigSendDuration["ID"] == "alice"]['Duration'].mean()

0.0009699940681457519

In [20]:
ModelConfigSendDuration[ModelConfigSendDuration["ID"] == "bob"]['Duration'].mean()

0.0009421586990356445

In [21]:
ModelConfigSendDuration[ModelConfigSendDuration["ID"] == "charlie"]['Duration'].mean()

0.0009432077407836914

## GlobalInformationSend Duration

In [22]:
def get_coord_global_send_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("GlobalInformationSend", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [23]:
hpc1_gs_df = get_coord_global_send_duration(coord_df, "alice")

In [24]:
hpc2_gs_df = get_coord_global_send_duration(coord_df, "bob")

In [25]:
hpc3_gs_df = get_coord_global_send_duration(coord_df, "charlie")

In [26]:
hpc1_gs_df['Time'].mean()

0.3133407115936279

In [27]:
hpc2_gs_df['Time'].mean()

0.3175040721893311

In [28]:
hpc3_gs_df['Time'].mean()

0.32024874687194826

## Send Time

In [29]:
## EventType:
##     * GlobalModelSend
##     * LossFuncSend
##     * ModelConfigSend
def calculate_send_time(coord_df, worker_dfs, event_type):
    event_start_df = coord_df[(coord_df["EventType"] == event_type) & (coord_df["EventState"] == "start")]
    worker_event_send_time_dfs = {}
    
    for worker_id in worker_dfs.keys():
        worker_event_start_df = event_start_df[event_start_df["ID"] == worker_id]
        worker_event_start_df = worker_event_start_df.reset_index()
        del worker_event_start_df['index']
        worker_event_start_df.columns = ["EventType", "EventState", "ID", "StartTime"]
        
        worker_df = worker_dfs[worker_id]
        worker_event_recv_df = worker_df[worker_df["EventType"] == event_type]
        worker_event_recv_df = worker_event_recv_df.reset_index()
        del worker_event_recv_df['index']
        worker_event_recv_df.columns = ["WorkerID", "EventType", "EventState", "ID", "RecvTime"]
        
        worker_recv_time_series = worker_event_recv_df["RecvTime"]
        worker_event_send_time_df = pd.concat([worker_event_start_df, worker_recv_time_series], axis=1)
        worker_event_send_time_df["SendTime"] = worker_event_send_time_df.apply(lambda x: x["RecvTime"] - x["StartTime"], axis=1)
        del worker_event_send_time_df["EventState"]
        
        worker_event_send_time_dfs[worker_id] = worker_event_send_time_df
        
    return worker_event_send_time_dfs

In [30]:
worker_gms_send_time_dfs = calculate_send_time(coord_df, worker_dfs, "GlobalModelSend")

In [31]:
worker_gms_send_time_dfs['alice']['SendTime'].mean()

0.31011366844177246

In [32]:
worker_gms_send_time_dfs['alice']

,EventType,ID,StartTime,RecvTime,SendTime
0,GlobalModelSend,alice,1.598988e+09,1.598988e+09,0.361677
1,GlobalModelSend,alice,1.598988e+09,1.598988e+09,0.296891
2,GlobalModelSend,alice,1.598988e+09,1.598988e+09,0.315485
3,GlobalModelSend,alice,1.598988e+09,1.598988e+09,0.301552
4,GlobalModelSend,alice,1.598988e+09,1.598988e+09,0.336071
5,GlobalModelSend,alice,1.598988e+09,1.598988e+09,0.307584
6,GlobalModelSend,alice,1.598988e+09,1.598988e+09,0.309776
7,GlobalModelSend,alice,1.598988e+09,1.598988e+09,0.306883
8,GlobalModelSend,alice,1.598988e+09,1.598988e+09,0.310315
9,GlobalModelSend,alice,1.598988e+09,1.598988e+09,0.315312


In [33]:
worker_gms_send_time_dfs['charlie']['SendTime'].mean()

0.31753212213516235

In [34]:
# coord_df = read_trace(trace_dir, coord_trace_file)
# hpc1_df = read_trace(trace_dir, hpc1_trace_file, "alice")
# hpc2_df = read_trace(trace_dir, hpc2_trace_file, "bob")
# hpc3_df = read_trace(trace_dir, hpc3_trace_file, "charlie")

## LocalTraining Duration

In [35]:
hpc1_lt_df = get_worker_duration("alice", worker_dfs, "LocalTraining")

In [36]:
hpc2_lt_df = get_worker_duration("bob", worker_dfs, "LocalTraining")

In [37]:
hpc3_lt_df = get_worker_duration("charlie", worker_dfs, "LocalTraining")

In [38]:
hpc1_lt_df['Time'].mean()

13.574540662765504

In [39]:
hpc2_lt_df['Time'].mean()

9.33016802072525

In [40]:
hpc3_lt_df['Time'].mean()

10.005344891548157

In [41]:
hpc1_lt_df

,WorkerID,EventType,EventState,Time
0,alice,LocalTraining,duration,13.436776
1,alice,LocalTraining,duration,13.715997
2,alice,LocalTraining,duration,13.842978
3,alice,LocalTraining,duration,13.538459
4,alice,LocalTraining,duration,13.876474
5,alice,LocalTraining,duration,13.841714
6,alice,LocalTraining,duration,13.838154
7,alice,LocalTraining,duration,12.937228
8,alice,LocalTraining,duration,13.488678
9,alice,LocalTraining,duration,13.137236


## FitSagg Duration

In [42]:
def get_coord_fitsagg_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("FitSagg", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [43]:
hpc1_fs_df = get_coord_fitsagg_duration(coord_df, "alice")

In [44]:
hpc2_fs_df = get_coord_fitsagg_duration(coord_df, "bob")

In [45]:
hpc3_fs_df = get_coord_fitsagg_duration(coord_df, "charlie")

In [46]:
hpc1_fs_df['Time'].mean()

16.672366213798522

In [47]:
hpc2_fs_df['Time'].mean()

16.675291442871092

In [48]:
hpc3_fs_df['Time'].mean()

16.664860916137695

## EncryptMultiply Duration

In [49]:
hpc1_em_df = get_worker_duration("alice", worker_dfs, "EncryptMultiply")

In [50]:
hpc2_em_df = get_worker_duration("bob", worker_dfs, "EncryptMultiply")

In [51]:
hpc3_em_df = get_worker_duration("charlie", worker_dfs, "EncryptMultiply")

In [52]:
hpc1_em_df['Time'].mean()

2.7414315342903137

In [53]:
hpc2_em_df['Time'].mean()

6.980119740962982

In [54]:
hpc3_em_df['Time'].mean()

6.294533228874206

In [55]:
hpc1_em_df

,WorkerID,EventType,EventState,Time
0,alice,EncryptMultiply,duration,2.655156
1,alice,EncryptMultiply,duration,2.488327
2,alice,EncryptMultiply,duration,2.495913
3,alice,EncryptMultiply,duration,3.104249
4,alice,EncryptMultiply,duration,3.039503
5,alice,EncryptMultiply,duration,3.137661
6,alice,EncryptMultiply,duration,3.068573
7,alice,EncryptMultiply,duration,2.553304
8,alice,EncryptMultiply,duration,2.505005
9,alice,EncryptMultiply,duration,3.034689


In [56]:
hpc3_em_df

,WorkerID,EventType,EventState,Time
0,charlie,EncryptMultiply,duration,5.838547
1,charlie,EncryptMultiply,duration,6.201904
2,charlie,EncryptMultiply,duration,6.667344
3,charlie,EncryptMultiply,duration,6.498047
4,charlie,EncryptMultiply,duration,6.964555
5,charlie,EncryptMultiply,duration,7.083305
6,charlie,EncryptMultiply,duration,6.509825
7,charlie,EncryptMultiply,duration,5.604012
8,charlie,EncryptMultiply,duration,6.116922
9,charlie,EncryptMultiply,duration,5.742249


## EncryptParameter Duration

In [57]:
def get_encrypt_parameter_duration(worker_id, worker_dfs):
    worker_df = worker_dfs[worker_id]
    worker_ep_df = worker_df[worker_df["EventType"].str.contains('EncryptParameter', regex=False)]
    worker_ep_df = worker_ep_df.reset_index()
    
    del worker_ep_df["index"]
    del worker_ep_df["ID"]
    
    return worker_ep_df
    

In [58]:
# hpc1_ep_df = get_encrypt_parameter_duration("alice", worker_dfs)
hpc1_ep_df = get_worker_duration("alice", worker_dfs, "EncryptParameter")
hpc2_ep_df = get_worker_duration("bob", worker_dfs, "EncryptParameter")
hpc3_ep_df = get_worker_duration("charlie", worker_dfs, "EncryptParameter")

In [59]:
hpc1_ep_df[0:8]

,WorkerID,EventType,EventState,Time
0,alice,EncryptParameter0,duration,0.008208
1,alice,EncryptParameter1,duration,0.007254
2,alice,EncryptParameter2,duration,0.096836
3,alice,EncryptParameter3,duration,0.005957
4,alice,EncryptParameter4,duration,2.371753
5,alice,EncryptParameter5,duration,0.015386
6,alice,EncryptParameter6,duration,0.022224
7,alice,EncryptParameter7,duration,0.004442


In [60]:
mean_values = []
for i in range(8):
    mean_value = hpc1_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
    mean_values.append(mean_value)
    print(str(i)+":", mean_value)

0: 0.008632922172546386
1: 0.004727673530578613
2: 0.11859222650527954
3: 0.03444761037826538
4: 2.174571084976196
5: 0.14334408044815064
6: 0.09053375720977783
7: 0.009077227115631104


In [61]:
sum(mean_values)

2.5839265823364257

In [62]:
for i in range(8):
    mean_value = hpc2_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
    print(str(i)+":", mean_value)

0: 4.250299048423767
1: 0.006587338447570801
2: 0.14224008321762086
3: 0.07622889280319214
4: 2.0606893420219423
5: 0.10327472686767578
6: 0.09056872129440308
7: 0.006265199184417725


In [63]:
for i in range(8):
    mean_value = hpc3_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
    print(str(i)+":", mean_value)

0: 3.571883976459503
1: 0.004579067230224609
2: 0.1086920976638794
3: 0.03220844268798828
4: 1.700234878063202
5: 0.24840887784957885
6: 0.2065204381942749
7: 0.0079475998878479


In [64]:
hpc1_ep_df["Time"][0:8]

0    0.008208
1    0.007254
2    0.096836
3    0.005957
4    2.371753
5    0.015386
6    0.022224
7    0.004442
Name: Time, dtype: float64

In [65]:
hpc1_ep_df["Time"].mean()

0.3229908227920532

In [66]:
hpc2_ep_df["Time"].mean()

0.8420191690325737

In [67]:
hpc3_ep_df["Time"].mean()

0.7350594222545623

## MultiplyParameter Duration

In [68]:
def get_multiply_parameter_duration(worker_id, worker_dfs):
    worker_df = worker_dfs[worker_id]
    worker_ep_df = worker_df[worker_df["EventType"].str.contains('MultiplyParameter', regex=False)]
    worker_ep_df = worker_ep_df.reset_index()
    
    del worker_ep_df["index"]
    del worker_ep_df["ID"]
    
    return worker_ep_df

In [69]:
# hpc1_mp_df = get_multiply_parameter_duration("alice", worker_dfs)
hpc1_mp_df = get_worker_duration("alice", worker_dfs, "MultiplyParameter")
hpc2_mp_df = get_worker_duration("bob", worker_dfs, "MultiplyParameter")
hpc3_mp_df = get_worker_duration("charlie", worker_dfs, "MultiplyParameter")

In [70]:
hpc1_mp_df[:8]["Time"].sum()

0.11918282508850099

In [71]:
hpc1_mp_df["Time"].mean()

0.019214098155498505

In [72]:
hpc2_mp_df["Time"].mean()

0.029996877908706664

In [73]:
hpc3_mp_df["Time"].mean()

0.05132453441619873

## RetrieveTime duration

In [74]:
def get_coord_retrieve_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("RetrieveTime", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [75]:
hpc1_ret_df = get_coord_retrieve_duration(coord_df, "alice")

In [76]:
hpc2_ret_df = get_coord_retrieve_duration(coord_df, "bob")

In [77]:
hpc3_ret_df = get_coord_retrieve_duration(coord_df, "charlie")

In [78]:
hpc1_ret_df["Time"].mean()

0.01607893705368042

In [79]:
hpc2_ret_df["Time"].mean()

0.016192054748535155

In [80]:
hpc3_ret_df["Time"].mean()

0.018958091735839844

## ShareSend Duration

In [81]:
def calculate_share_send_duration(worker_id, worker_dfs, nr_parameters):
    worker_df = worker_dfs[worker_id]
    worker_send_share_df = worker_df[worker_df["EventType"].str.contains('SendShare', regex=False)]
    
    worker_send_share_start_df = worker_send_share_df[worker_send_share_df["EventState"] == 'start']
    worker_send_share_start_df = worker_send_share_start_df.reset_index()
    worker_send_share_start_df = worker_send_share_start_df[["WorkerID", "EventType", "ID", "Time"]]
    worker_send_share_start_df.columns = ["WorkerID", "EventType", "ID", "StartTime"]
    
    worker_send_share_end_df = worker_send_share_df[worker_send_share_df["EventState"] == 'end']
    worker_send_share_end_df = worker_send_share_end_df.reset_index()
    
    send_share_df_cat = pd.concat([worker_send_share_start_df, worker_send_share_end_df["Time"]], axis=1)
    send_share_df_cat.columns = ["WorkerID", "EventType", "ID", "StartTime", "EndTime"]
    send_share_df_cat["Duration"] = send_share_df_cat.apply(lambda x: x["EndTime"] - x["StartTime"], axis=1)
    
    parameter_index = []
    nr_worker = len(worker_dfs)
    p_index = -1

    for i in range(len(send_share_df_cat)):
        tmp_index = (i%nr_worker)
        if tmp_index == 0:
            p_index += 1
            p_index = p_index % nr_parameters
        parameter_index.append(p_index)
        
    send_share_df_cat.insert(2, "p_idx", parameter_index)
    obj_id_list = send_share_df_cat.apply(lambda x: x["EventType"].split("_")[1], axis=1)
    send_share_df_cat.insert(2, "obj_id", obj_id_list)
    
    del send_share_df_cat["EventType"]
    send_share_df_cat.insert(1, "EventType", "SendShare")
    
    return send_share_df_cat


In [82]:
alice_send_share_df_cat = calculate_share_send_duration("alice", worker_dfs, 8)
bob_send_share_df_cat = calculate_share_send_duration("bob", worker_dfs, 8)
charlie_send_share_df_cat = calculate_share_send_duration("charlie", worker_dfs, 8)

In [83]:
alice_send_share_df_cat

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,EndTime,Duration
0,alice,SendShare,23997006610,0,alice,1.598988e+09,1.598988e+09,0.001037
1,alice,SendShare,75182669587,0,bob,1.598988e+09,1.598988e+09,0.003384
2,alice,SendShare,5023164647,0,charlie,1.598988e+09,1.598988e+09,0.003255
3,alice,SendShare,80407731133,1,alice,1.598988e+09,1.598988e+09,0.000543
4,alice,SendShare,66576630703,1,bob,1.598988e+09,1.598988e+09,0.002545
...,...,...,...,...,...,...,...,...
475,alice,SendShare,8660112579,6,bob,1.598989e+09,1.598989e+09,0.014551
476,alice,SendShare,20585141147,6,charlie,1.598989e+09,1.598989e+09,0.011652
477,alice,SendShare,16580334952,7,alice,1.598989e+09,1.598989e+09,0.000613
478,alice,SendShare,20844511931,7,bob,1.598989e+09,1.598989e+09,0.002126


In [84]:
bob_send_share_df_cat

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,EndTime,Duration
0,bob,SendShare,9659550396,0,alice,1.598988e+09,1.598988e+09,4.018958
1,bob,SendShare,41166973424,0,bob,1.598988e+09,1.598988e+09,0.000987
2,bob,SendShare,58600117538,0,charlie,1.598988e+09,1.598988e+09,0.004365
3,bob,SendShare,28100465353,1,alice,1.598988e+09,1.598988e+09,0.002429
4,bob,SendShare,37063113338,1,bob,1.598988e+09,1.598988e+09,0.000874
...,...,...,...,...,...,...,...,...
475,bob,SendShare,22649445386,6,bob,1.598989e+09,1.598989e+09,0.000551
476,bob,SendShare,96922076436,6,charlie,1.598989e+09,1.598989e+09,0.009544
477,bob,SendShare,76620535,7,alice,1.598989e+09,1.598989e+09,0.002061
478,bob,SendShare,5931891331,7,bob,1.598989e+09,1.598989e+09,0.000813


In [85]:
charlie_send_share_df_cat

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,EndTime,Duration
0,charlie,SendShare,97943374482,0,alice,1.598988e+09,1.598988e+09,3.196364
1,charlie,SendShare,59551409359,0,bob,1.598988e+09,1.598988e+09,0.006402
2,charlie,SendShare,91895774378,0,charlie,1.598988e+09,1.598988e+09,0.000946
3,charlie,SendShare,19624518335,1,alice,1.598988e+09,1.598988e+09,0.001811
4,charlie,SendShare,88952952227,1,bob,1.598988e+09,1.598988e+09,0.001456
...,...,...,...,...,...,...,...,...
475,charlie,SendShare,56920097722,6,bob,1.598989e+09,1.598989e+09,0.012220
476,charlie,SendShare,13710880754,6,charlie,1.598989e+09,1.598989e+09,0.000418
477,charlie,SendShare,65150237747,7,alice,1.598989e+09,1.598989e+09,0.001266
478,charlie,SendShare,24243605242,7,bob,1.598989e+09,1.598989e+09,0.001101


## ShareSend Send Time

In [86]:
def calculate_share_send_time(sender_id, receiver_id, worker_dfs, nr_parameter):
    sender_df = worker_dfs[sender_id]
    receiver_df = worker_dfs[receiver_id]
    
    sender_send_share_df = sender_df[sender_df["EventType"].str.contains('SendShare', regex=False)]
    receiver_get_share_df = receiver_df[receiver_df["EventType"].str.contains('GetShare', regex=False)]
    
    send_share_start_df = sender_send_share_df[(sender_send_share_df["ID"] == receiver_id) & (sender_send_share_df["EventState"] == 'start')]
    get_share_df = receiver_get_share_df[receiver_get_share_df["ID"] == sender_id]
    send_share_start_df = send_share_start_df.reset_index()
    get_share_df = get_share_df.reset_index()
    
    del send_share_start_df['index']
    del send_share_start_df['EventState']
    
    send_time_df = pd.concat([send_share_start_df, get_share_df["Time"]], axis=1)
    send_time_df.columns = ["WorkerID", "EventType", "ID", "StartTime", "RecvTime"]
    send_time_df["SendTime"] = send_time_df.apply(lambda x: x["RecvTime"] - x["StartTime"], axis=1)
    
    nr_parameter = 8
    parameter_index = []

    for i in range(len(send_time_df)):
        tmp_index = (i%nr_parameter)
        parameter_index.append(tmp_index)
        
    send_time_df.insert(2, "p_idx", parameter_index)
    
    obj_id_list = send_time_df.apply(lambda x: x["EventType"].split("_")[1], axis=1)
    send_time_df.insert(2, "obj_id", obj_id_list)
    
    del send_time_df["EventType"]
    send_time_df.insert(1, "EventType", "SendShare")
    
    return send_time_df

In [87]:
send_time_df = calculate_share_send_time("alice", "bob", worker_dfs, 8)

In [88]:
send_time_df

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,RecvTime,SendTime
0,alice,SendShare,75182669587,0,bob,1.598988e+09,1.598988e+09,0.002891
1,alice,SendShare,66576630703,1,bob,1.598988e+09,1.598988e+09,0.002126
2,alice,SendShare,46212799352,2,bob,1.598988e+09,1.598988e+09,0.043050
3,alice,SendShare,79009060855,3,bob,1.598988e+09,1.598988e+09,0.002898
4,alice,SendShare,54346801011,4,bob,1.598988e+09,1.598988e+09,1.252096
...,...,...,...,...,...,...,...,...
155,alice,SendShare,76956495691,3,bob,1.598989e+09,1.598989e+09,0.001013
156,alice,SendShare,64148944601,4,bob,1.598989e+09,1.598989e+09,1.220871
157,alice,SendShare,445149044,5,bob,1.598989e+09,1.598989e+09,0.002263
158,alice,SendShare,8660112579,6,bob,1.598989e+09,1.598989e+09,0.012199


## AllWorkersTrainingTime Duration

In [89]:
coord_awtt_df = get_coord_duration(coord_df, "AllWorkersTrainingTime")

In [90]:
coord_awtt_df['Time'].mean()

16.693915832042695

In [91]:
coord_awtt_df

,EventType,EventState,Time
0,AllWorkersTrainingTime,duration,16.511965
1,AllWorkersTrainingTime,duration,16.590312
2,AllWorkersTrainingTime,duration,16.724470
3,AllWorkersTrainingTime,duration,17.008052
4,AllWorkersTrainingTime,duration,17.301688
5,AllWorkersTrainingTime,duration,17.346246
6,AllWorkersTrainingTime,duration,17.267323
7,AllWorkersTrainingTime,duration,15.868551
8,AllWorkersTrainingTime,duration,16.366584
9,AllWorkersTrainingTime,duration,16.535329


## AggregationTime

In [92]:
coord_aggt_df = get_coord_duration(coord_df, "AggregationTime")

In [93]:
coord_aggt_df["Time"].mean()

0.10683181285858154

In [94]:
coord_aggt_df

,EventType,EventState,Time
0,AggregationTime,duration,0.104095
1,AggregationTime,duration,0.107789
2,AggregationTime,duration,0.105432
3,AggregationTime,duration,0.107143
4,AggregationTime,duration,0.107304
5,AggregationTime,duration,0.106201
6,AggregationTime,duration,0.107841
7,AggregationTime,duration,0.103993
8,AggregationTime,duration,0.107758
9,AggregationTime,duration,0.106346


## DecryptionTime

In [95]:
coord_decrypt_df = get_coord_duration(coord_df, "DecryptionTime")

In [96]:
coord_decrypt_df['Time'].mean()

0.1773815393447876

In [97]:
coord_decrypt_df

,EventType,EventState,Time
0,DecryptionTime,duration,0.183629
1,DecryptionTime,duration,0.174788
2,DecryptionTime,duration,0.176217
3,DecryptionTime,duration,0.178724
4,DecryptionTime,duration,0.197265
5,DecryptionTime,duration,0.183573
6,DecryptionTime,duration,0.177899
7,DecryptionTime,duration,0.187322
8,DecryptionTime,duration,0.175537
9,DecryptionTime,duration,0.175527


## RoundTime

In [98]:
coord_round_time_df = get_coord_duration(coord_df, "RoundTime")

In [99]:
coord_round_time_df["Time"].mean()

17.017746257781983

In [100]:
coord_round_time_df

,EventType,EventState,Time
0,RoundTime,duration,16.799823
1,RoundTime,duration,16.914645
2,RoundTime,duration,17.047378
3,RoundTime,duration,17.335857
4,RoundTime,duration,17.649704
5,RoundTime,duration,17.678650
6,RoundTime,duration,17.594516
7,RoundTime,duration,16.201091
8,RoundTime,duration,16.692443
9,RoundTime,duration,16.859439
